In [5]:
import spacy

import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
import json

from spacy.language import Language
from typing import Dict, List, Set, Tuple
from collections import defaultdict, namedtuple
import copy
from statistics import mean 
import io
from PIL import Image, ImageDraw, ImageFont
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from google.protobuf.json_format import MessageToJson, Parse
from google.protobuf import json_format

import os
import sys
sys.path.append("..")
from models.dish_segmenter import Dish
from models.bounding_box import FeatureType, Point, BoundingBox, DSU
from models.word_unit import WordUnit
from utils.cv_preprocess import *
from utils.file_utils import *
from utils.nlp_preprocess import *
from IPython.display import display


In [6]:
filein = '../downloaded_menu/cleaned_img/5._Ma_La_Sichuan_0_page_5.png'
base_name = os.path.basename(filein)
file_name_without_extension = os.path.splitext(base_name)[0]


raw_ocr_directory = '../downloaded_menu/raw_ocr/'
raw_ocr_filename = file_name_without_extension + "_raw_annotation.json"
raw_ocr_path = os.path.join(raw_ocr_directory, raw_ocr_filename)

preprocessed_ocr_directory = '../downloaded_menu/prep_ocr/'
preprocessed_ocr_filename = file_name_without_extension + "_prep_ocr.json"
preprocessed_ocr_path = os.path.join(preprocessed_ocr_directory, preprocessed_ocr_filename)

# Open the file for reading
with open(raw_ocr_path, 'r') as file:
    document = json.load(file)

In [8]:

def process_menu(filein):
    base_name = os.path.basename(filein)
    file_name_without_extension = os.path.splitext(base_name)[0]

    raw_ocr_directory = '../downloaded_menu/raw_ocr/'
    raw_ocr_filename = file_name_without_extension + "_raw_annotation.json"
    raw_ocr_path = os.path.join(raw_ocr_directory, raw_ocr_filename)

    document = load_json(raw_ocr_path)
    image = Image.open(filein)

    bounds = process_bounds_in_paragraph(document)
    filtered_bounds, chinese_bbox, english_bbox = filter_and_classify_bounds(bounds)

    container_width, container_height = image.size
    extended_boxes = extend_bounding_boxes(chinese_bbox, container_width, container_height)
    
    sorted_bounding_boxes = sorted(filtered_bounds, key=lambda bbox: (bbox.y_min, bbox.x_min))
    grouped_list = group_extended_boxes(extended_boxes, sorted_bounding_boxes)
    print(grouped_list)
    for string_list in grouped_list:
        
        dish = segment_dish_text_list(string_list)
        print(dish)

# Usage example
filein = '../downloaded_menu/cleaned_img/1._Gouqi_London_0_page_1.png'
process_menu(filein)


[[['唐', '舍'], ['GOUQI']]]
Chinese Name: ['唐', '舍']
English Name: ['GOUQI']
Chinese Description: []
English Description: []
Raw text: None

